### Notebook to format dates in the point 46, 47, 48 csv files, to avoid dates being in different formats (all in classic format dd/mm/yyyy)

In [14]:
import os 

path = "C:/Users/alexa/2A/MOLONARI1D/dataAnalysis/data_traite"
os.listdir(path)[2:]

L=['point46_pression_traité.csv', 'point46_temperature_traité.csv', 'point47_pression_traité.csv', 'point47_temperature_traité.csv', 'point48_pression_traité.csv', 'point48_temperature_traité.csv']

for file in L:
    with open(path+'/'+file, 'r') as f:
        data = f.readlines()
        f.close()
    with open('new' + file, 'w') as new_file:
        new_file.write(data[0])
        new_file.write(data[1])
        for i, line in enumerate(data[2:]):
            if k==0:
                if int(line[0:2]) == int(data[i+1][0:2]) or int(line[0:2]) == int(data[i+1][0:2]) + 1:
                    new_file.write(line)
                else:
                    k=1
                    nd = line[3:5]
                    nm = line[0:2]
                    line = nd + line[2] + nm + line[5:]
                    new_file.write(line)
                    data[i+2] = line
            elif k==1:
                nd = line[3:5]
                nm = line[0:2]
                if int(line[3:5])==int(data[i+1][0:2]) or int(line[3:5])==int(data[i+1][0:2])+1:
                    line = nd + line[2] + nm + line[5:]
                    new_file.write(line)
                    data[i+2] = line
                else:
                    k=0
                    new_file.write(line)


6
